<a href="https://colab.research.google.com/github/sidhu2690/Geometric-DL/blob/main/Node2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import numpy as np
import random

# Graph as adjacency list
graph = {
    0: [1,3],
    1: [0,2],
    2: [1,3],
    3: [0,2]
}

def random_walk(start, walk_length=5):
    walk = [start]
    for _ in range(walk_length-1):
        cur = walk[-1]
        nxt = random.choice(graph[cur])
        walk.append(nxt)
    return walk

# Example walk
print(random_walk(0))


[0, 3, 2, 1, 2]


In [91]:
def generate_pairs(walk, window_size=2):
    pairs = []
    for i, center in enumerate(walk):
        for j in range(max(0, i-window_size), min(len(walk), i+window_size+1)):
            if i != j:
                pairs.append((center, walk[j]))
    return pairs

walk = random_walk(0)
pairs = generate_pairs(walk)
print("Random Walk:", walk)
print("Training Pairs:", pairs)


Random Walk: [0, 3, 0, 3, 0]
Training Pairs: [(0, 3), (0, 0), (3, 0), (3, 0), (3, 3), (0, 0), (0, 3), (0, 3), (0, 0), (3, 3), (3, 0), (3, 0), (0, 0), (0, 3)]


In [92]:
num_nodes = 4
embed_dim = 3  # size of z
Z = np.random.randn(num_nodes, embed_dim)

In [93]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def train_pair(center, context, Z, lr=0.01, k=2):
    # Positive pair
    z_u = Z[center]
    z_v = Z[context]

    # Positive update
    score = sigmoid(np.dot(z_u, z_v))
    grad = (1 - score)
    Z[center] += lr * grad * z_v
    Z[context] += lr * grad * z_u

    # Negative samples
    for _ in range(k):
        neg = random.randint(0, num_nodes-1)
        if neg == center: continue
        z_n = Z[neg]
        score_neg = sigmoid(np.dot(z_u, z_n))
        grad_neg = -score_neg
        Z[center] += lr * grad_neg * z_n
        Z[neg] += lr * grad_neg * z_u

    return Z


In [94]:
# Train
for epoch in range(100):
    for node in graph.keys():
        walk = random_walk(node, walk_length=5)
        pairs = generate_pairs(walk, window_size=2)
        for center, context in pairs:
            Z = train_pair(center, context, Z)

print("Final embeddings (Z):")
print(Z)


Final embeddings (Z):
[[ 1.04977213  2.45174522  0.55268559]
 [ 0.47224105 -0.32650802 -0.88530871]
 [-1.66079528  0.29864532 -0.45937652]
 [-0.03256875 -0.61807368  1.47605242]]
